In [2]:
import os
import torch
import numpy as np
import joblib
import requests
from transformers import BertTokenizer, BertModel
from tensorflow.keras.models import load_model

# Ensure GPU is used if available
device = "cuda" if torch.cuda.is_available() else "cpu"

##############################
# 1. Helper Function to Download Files
##############################
GITHUB_BASE_URL = "https://raw.githubusercontent.com/rixprakash/Jarheads/main/Project%201/OUTPUT/"

FILES_TO_DOWNLOAD = [
    "logistic_regression_bert.pkl",
    "neural_network_bert.h5",
    "label_encoder.pkl"
]

def download_file(filename):
    """Downloads a file from the GitHub repository if it's missing."""
    if not os.path.exists(filename):
        print(f"Downloading {filename} from GitHub...")
        url = GITHUB_BASE_URL + filename
        response = requests.get(url)
        if response.status_code == 200:
            with open(filename, "wb") as f:
                f.write(response.content)
            print(f"{filename} downloaded successfully!")
        else:
            raise FileNotFoundError(f"Failed to download {filename}")

# Ensure all required files are present
for file in FILES_TO_DOWNLOAD:
    download_file(file)

##############################
# 2. Load Pretrained BERT Model
##############################
print("Loading BERT model...")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased").to(device)
print("BERT model loaded successfully!")

##############################
# 3. Load Trained Models & Label Encoder
##############################
print("Loading trained models...")
logistic_model = joblib.load("logistic_regression_bert.pkl")
nn_model = load_model("neural_network_bert.h5")
label_encoder = joblib.load("label_encoder.pkl")
print("Models loaded successfully!")

##############################
# 4. Function to Convert Lyrics to BERT Embeddings
##############################
def get_bert_embedding(text):
    """Converts a single piece of text into a BERT embedding."""
    tokens = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    tokens = {key: val.to(device) for key, val in tokens.items()}

    with torch.no_grad():
        output = bert_model(**tokens)

    embedding = output.last_hidden_state[:, 0, :].cpu().numpy()
    return embedding

##############################
# 5. Predict Genre Function
##############################
def predict_genre(lyrics):
    """Predicts the genre of given lyrics."""
    print("\nProcessing lyrics...")
    embedding = get_bert_embedding(lyrics)

    # Predict with Logistic Regression
    logreg_pred = logistic_model.predict(embedding)[0]
    logreg_genre = label_encoder.inverse_transform([logreg_pred])[0]

    # Predict with Neural Network
    nn_pred = nn_model.predict(embedding)
    nn_genre = label_encoder.inverse_transform([np.argmax(nn_pred)])[0]

    print("\n🎵 Predicted Genres 🎵")
    print(f"🎼 Logistic Regression: {logreg_genre}")
    print(f"🎵 Neural Network: {nn_genre}")
    return logreg_genre, nn_genre

##############################
# 6. Run the Prediction
##############################
if __name__ == "__main__":
    sample_lyrics = "I'm walking down the lonely road, with a guitar in my hand, singing the blues."
    predict_genre(sample_lyrics)


Loading BERT model...


BERT model loaded successfully!
Loading trained models...
Models loaded successfully!

Processing lyrics...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step

🎵 Predicted Genres 🎵
🎼 Logistic Regression: rock
🎵 Neural Network: pop
